# --- Day 16: Chronal Classification ---

As you see the Elves defend their hot chocolate successfully, you go back to falling through time. This is going to become a problem.

If you're ever going to return to your own time, you need to understand how this device on your wrist works. You have a little while before you reach your next destination, and with a bit of trial and error, you manage to pull up a programming manual on the device's tiny screen.

According to the manual, the device has four registers (numbered 0 through 3) that can be manipulated by instructions containing one of 16 opcodes. The registers start with the value 0.

Every instruction consists of four values: an opcode, two inputs (named A and B), and an output (named C), in that order. The opcode specifies the behavior of the instruction and how the inputs are interpreted. The output, C, is always treated as a register.

In the opcode descriptions below, if something says "value A", it means to take the number given as A literally. (This is also called an "immediate" value.) If something says "register A", it means to use the number given as A to read from (or write to) the register with that number. So, if the opcode addi adds register A and value B, storing the result in register C, and the instruction addi 0 7 3 is encountered, it would add 7 to the value contained by register 0 and store the sum in register 3, never modifying registers 0, 1, or 2 in the process.

Many opcodes are similar except for how they interpret their arguments. The opcodes fall into seven general categories:

Addition:

    addr (add register) stores into register C the result of adding register A and register B.
    addi (add immediate) stores into register C the result of adding register A and value B.

Multiplication:

    mulr (multiply register) stores into register C the result of multiplying register A and register B.
    muli (multiply immediate) stores into register C the result of multiplying register A and value B.

Bitwise AND:

    banr (bitwise AND register) stores into register C the result of the bitwise AND of register A and register B.
    bani (bitwise AND immediate) stores into register C the result of the bitwise AND of register A and value B.

Bitwise OR:

    borr (bitwise OR register) stores into register C the result of the bitwise OR of register A and register B.
    bori (bitwise OR immediate) stores into register C the result of the bitwise OR of register A and value B.

Assignment:

    setr (set register) copies the contents of register A into register C. (Input B is ignored.)
    seti (set immediate) stores value A into register C. (Input B is ignored.)

Greater-than testing:

    gtir (greater-than immediate/register) sets register C to 1 if value A is greater than register B. Otherwise, register C is set to 0.
    gtri (greater-than register/immediate) sets register C to 1 if register A is greater than value B. Otherwise, register C is set to 0.
    gtrr (greater-than register/register) sets register C to 1 if register A is greater than register B. Otherwise, register C is set to 0.

Equality testing:

    eqir (equal immediate/register) sets register C to 1 if value A is equal to register B. Otherwise, register C is set to 0.
    eqri (equal register/immediate) sets register C to 1 if register A is equal to value B. Otherwise, register C is set to 0.
    eqrr (equal register/register) sets register C to 1 if register A is equal to register B. Otherwise, register C is set to 0.

Unfortunately, while the manual gives the name of each opcode, it doesn't seem to indicate the number. However, you can monitor the CPU to see the contents of the registers before and after instructions are executed to try to work them out. Each opcode has a number from 0 through 15, but the manual doesn't say which is which. For example, suppose you capture the following sample:
```
Before: [3, 2, 1, 1]
9 2 1 2
After:  [3, 2, 2, 1]
```
This sample shows the effect of the instruction 9 2 1 2 on the registers. Before the instruction is executed, register 0 has value 3, register 1 has value 2, and registers 2 and 3 have value 1. After the instruction is executed, register 2's value becomes 2.

The instruction itself, 9 2 1 2, means that opcode 9 was executed with A=2, B=1, and C=2. Opcode 9 could be any of the 16 opcodes listed above, but only three of them behave in a way that would cause the result shown in the sample:

    Opcode 9 could be mulr: register 2 (which has a value of 1) times register 1 (which has a value of 2) produces 2, which matches the value stored in the output register, register 2.
    Opcode 9 could be addi: register 2 (which has a value of 1) plus value 1 produces 2, which matches the value stored in the output register, register 2.
    Opcode 9 could be seti: value 2 matches the value stored in the output register, register 2; the number given for B is irrelevant.

None of the other opcodes produce the result captured in the sample. Because of this, the sample above behaves like three opcodes.

You collect many of these samples (the first section of your puzzle input). The manual also includes a small test program (the second section of your puzzle input) - you can ignore it for now.

**Ignoring the opcode numbers, how many samples in your puzzle input behave like three or more opcodes?**


# Part 1

**N.B. I completed day 19 before doing day 16, so I will use the code from that, extracted as the day19 library, to solve day16**

In [1]:
from day19 import *
from collections import Counter
import re

In [2]:
data = open('input.txt').read().splitlines()

In [3]:
def parsedata(filedata):
    triples = []
    program = []
    rest = filedata[:]
    while len(rest):
        first, *rest = rest
        if first.startswith('Before:'):
            second, third, _, *rest = rest # _ to skip the blank line after each sample block
            first = list(map(int, re.findall(r'(-?\d+)', first)))
            second = list(map(int, re.findall(r'(-?\d+)', second)))
            third = list(map(int, re.findall(r'(-?\d+)', third)))
            triples.append([first, second, third])
        else:
            line = list(map(int, re.findall(r'(-?\d+)', first)))
            if len(line):
                program.append(line)
    return triples, program
        

In [4]:
triples, program = parsedata(data)

In [5]:
vm = VM(prog=['nop'], nregisters=4)
c = Counter()
matchedlist = []
for line in triples:
    before, prog, after = line
    opcode, A, B, C = prog
    matchcount = 0
    matches = []
    for instr in INSTRS:
        vm.ip = 0
        vm._set_registers(before)
        vm.prog = [f'{instr} {A} {B} {C}']
        vm.step()
        if vm.registers == after:
            matchcount += 1
            matches.append(instr)
    matchedlist.append([opcode, matchcount, matches])

In [6]:
print(f'Part 1 answer: there are {len([i for i in matchedlist if i[1] >= 3])} samples that match 3 or more opcodes')

Part 1 answer: there are 580 samples that match 3 or more opcodes


# Part 2
Hypothesis: There will be at least one opcode with only one possibility.  We can start with that/those and eliminate those instructions from the remaining possible lists.  Iterating this process will give us the pammings.

In [7]:
[i for i in matchedlist if i[1] == 1][:5]

[[11, 1, ['eqri']],
 [11, 1, ['eqri']],
 [11, 1, ['eqri']],
 [11, 1, ['eqri']],
 [11, 1, ['eqri']]]

In [8]:
def eliminate(datalist, instr):
    newl = []
    for l in datalist:
        opcode, count, instrlist = l
        if instr in instrlist:
            instrlist.remove(instr)
            count = count - 1
            if count == 0:
                continue
        newl.append([opcode, count, instrlist])
    return newl

In [9]:
mappings = {}
while len(matchedlist):
    el = [i for i in matchedlist if i[1] == 1][0] # lets assume there's one at each stage
    opcode, c, instrl = el
    mappings[opcode] = instrl[0]
    matchedlist = eliminate(matchedlist, instrl[0])
    print(f'The set of all lengths is now: {set(x[1] for x in matchedlist)} from {len(matchedlist)} entries')

The set of all lengths is now: {1, 2, 3, 4, 5, 6, 7, 8, 9, 13} from 725 entries
The set of all lengths is now: {1, 2, 3, 4, 5, 6, 7, 8, 12} from 679 entries
The set of all lengths is now: {1, 2, 3, 4, 5, 6, 7, 11} from 629 entries
The set of all lengths is now: {1, 2, 3, 4, 5, 6, 7, 11} from 578 entries
The set of all lengths is now: {1, 2, 3, 4, 6, 7, 11} from 528 entries
The set of all lengths is now: {1, 2, 3, 4, 5, 6, 10} from 479 entries
The set of all lengths is now: {1, 2, 3, 4, 5, 9} from 430 entries
The set of all lengths is now: {1, 2, 4, 5, 8} from 376 entries
The set of all lengths is now: {1, 3, 4, 5, 7} from 328 entries
The set of all lengths is now: {1, 2, 4, 6} from 283 entries
The set of all lengths is now: {1, 2, 3, 5} from 232 entries
The set of all lengths is now: {1, 2, 4} from 187 entries
The set of all lengths is now: {1, 3} from 145 entries
The set of all lengths is now: {1, 2} from 97 entries
The set of all lengths is now: {1} from 44 entries
The set of all len

In [10]:
newprogram = []
for line in program:
    opcode, A, B, C = line
    newprogram.append(f'{mappings[opcode]} {A} {B} {C}')

In [11]:
vm = VM(prog=newprogram, nregisters=4)

In [12]:
print(f'Part two answer: at the end of the program, register 0 contains {vm.run()}')

Finished execution of program at address 919 after 919 steps. Registers are [537, 2, 3, 2].
Part two answer: at the end of the program, register 0 contains 537
